In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path_to_root = '/content/drive/My Drive/Colab Notebooks/BatuEl_Dissertation'
sys.path.append(path_to_root)
print("Drive mounted.")

data_path = path_to_root + '/data'
models_path = path_to_root + '/models'
representations_path = path_to_root + '/representations'

Mounted at /content/drive
Drive mounted.


In [ ]:
import torch
import tqdm
from reprshift.learning.algorithms import ERM
from reprshift.models.hparams import hparams_f, HPARAM_SEARCH
from reprshift.dataset.datasets import MultiNLI
from reprshift.dataset.dataloaders import InfiniteDataLoader, FastDataLoader

In [ ]:
hparams = hparams_f('ERM')
learning_rates = [lr for lr in HPARAM_SEARCH['lr']]
hparams, learning_rates

({'batch_size': 32,
  'last_layer_dropout': 0.5,
  'optimizer': 'adamw',
  'weight_decay': 0.0001,
  'lr': 1e-05,
  'group_balanced': False,
  'num_training_steps': 30001,
  'num_warmup_steps': 0},
 [1e-06, 1e-05, 0.0001])

In [7]:
device = "cuda"
train_weights = None
batch_size = hparams['batch_size']

train_dataset = MultiNLI(data_path, 'tr', hparams)
train_loader = InfiniteDataLoader(  dataset=train_dataset,
                                    weights=train_weights,
                                    batch_size=batch_size,
                                    num_workers=1)
steps_per_epoch = len(train_dataset) / batch_size

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [8]:
algorithm_name = 'ERM'
random_seeds = [0,1,2,3,4]
init_state_dict_path = lambda random_seed : models_path + f'/00_randominit/seed{random_seed}/sd_epoch0.pth'
state_dict_PATH = models_path + '/01_erm/hpselection/'
init_state_dict_path(0), state_dict_PATH

('/content/drive/My Drive/Colab Notebooks/BatuEl_Dissertation/models/00_randominit/seed0/sd_epoch0.pth',
 '/content/drive/My Drive/Colab Notebooks/BatuEl_Dissertation/models/01_erm/hpselection/')

In [9]:
[learning_rates[2]]

[0.0001]

In [10]:
start_step = 1
n_steps = 30001 #30001 #hparams['num_training_steps']
checkpoint_freq = 1000
train_losses = {}

seed = 0

for lr in [learning_rates[2]]:
    hparams['lr'] = lr
    print('Training Learning Rate:', hparams['lr'])
    algorithm = ERM(num_classes=3, num_attributes=2, hparams=hparams)
    algorithm.to(device)
    algorithm.load_state_dict(torch.load(init_state_dict_path(seed)))
    train_losses[seed] = []

    train_loader = InfiniteDataLoader(  dataset=train_dataset,
                                        weights=train_weights,
                                        batch_size=batch_size,
                                        num_workers=1)
    train_minibatches_iterator = iter(train_loader)

    for step in tqdm.tqdm(range(start_step, n_steps)):
        ### Training Step ###
        i, x, y, a = next(train_minibatches_iterator)
        minibatch_device = (i, x.to(device), y.to(device), a.to(device))
        algorithm.train()
        step_vals = algorithm.update(minibatch_device, step)
        train_losses[seed].append(step_vals['loss'])

        ### Evaluation ###
        if (step % checkpoint_freq == 0) or (step == n_steps - 1):
            epoch = int(step / checkpoint_freq)
            algorithm_state_dict = algorithm.state_dict()
            algorithm_state_dict_PATH = state_dict_PATH + f'lr{lr}/sd_epoch{epoch}.pth'
            torch.save(algorithm_state_dict, algorithm_state_dict_PATH)

    loss_PATH = state_dict_PATH + f'Loss_{algorithm_name}_{seed}.pth'
    torch.save(train_losses, loss_PATH)

Training Learning Rate: 0.0001


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 30000/30000 [1:22:38<00:00,  6.05it/s]
